<a href="https://colab.research.google.com/github/mustapha786/BrightFieldImageDefectDetection/blob/main/vgg16(448_448).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models, layers, optimizers
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Connect Google Colab to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define your data paths in Google Drive
drive_path = '/content/drive/MyDrive/Colab Notebooks'
train_data_dir = os.path.join(drive_path, 'Train')
test_data_dir = os.path.join(drive_path, 'Test')



img_width, img_height = 224, 224
batch_size = 32
epochs = 50  # Increase the number of epochs
learning_rate = 1e-5

# Data augmentation for the training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Rescaling for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Training data generator
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# Test data generator
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='binary')

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Unfreeze more layers in the base model
for layer in base_model.layers[:-8]:
    layer.trainable = True

# Create a new model with additional layers
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // batch_size,
                    callbacks=[early_stopping])

# Evaluate the model on the test set
predictions = model.predict(test_generator)
y_pred = np.round(predictions)
y_true = test_generator.classes

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculate recall
recall = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
print(f'Recall: {recall}')

# Classification report
class_report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print(f'Classification Report:\n{class_report}')

# Plot training history
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Mounted at /content/drive
Found 471 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
58889256/58889256 [==============================] - 1s 0us/step
Epoch 1/50
 1/14 [=>............................] - ETA: 17:24 - loss: 1.0590 - accuracy: 0.4688